In [26]:
import json
import wave  
import pyaudio  
import os  
import numpy as np  
from pydub import AudioSegment  # Убедитесь, что библиотека pydub установлена  
from vosk import Model, KaldiRecognizer  
from langdetect import detect, DetectorFactory  
import librosa  
import ffmpeg
import sounddevice as sd  
import time 
ffmpeg_path = f"ffmpeg" # Full path to bin folder
os.environ["PATH"] += os.pathsep + ffmpeg_path


c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# VOSK

In [11]:

WAVE_OUTPUT_FILENAME = "output.wav"  # Имя выходного файла  
model = Model('v')
 

Запись внизу


## Тест ввода через файл

In [ ]:
def record_audio_vosk(output_filename):  
    """Записывает аудио с микрофона в WAV файл."""  
    CHUNK = 1024  # Количество фреймов за один раз  
    FORMAT = pyaudio.paInt32  # Формат звука (16-битный целочисленный)  
    CHANNELS = 1  # Количество каналов (1 - моно, 2 - стерео)  
    RATE = 44100  # Частота дискретизации (в Гц)  

    p = pyaudio.PyAudio()  
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)  

    print("Запись... (нажмите Ctrl+C для остановки)")  

    frames = []  # Список для хранения фреймов аудио  

    try:  
        while True:  
            data = stream.read(CHUNK)  
            frames.append(data)  # Добавление данных в список  

    except KeyboardInterrupt:  
        print("Остановка записи...")  

    stream.stop_stream()  
    stream.close()  
    p.terminate()  

    with wave.open(output_filename, 'wb') as wf:  
        wf.setnchannels(CHANNELS)  
        wf.setsampwidth(p.get_sample_size(FORMAT))  
        wf.setframerate(RATE)  
        wf.writeframes(b''.join(frames))  # Запись всех фреймов в файл  

    print(f"Запись сохранена в {output_filename}")  

def ogg_to_pcm_array(ogg_file):  
    """Конвертирует ogg файл в PCM массив."""  
    if not os.path.exists(ogg_file):  
        raise FileNotFoundError(f"Файл не найден: {ogg_file}")  

    audio = AudioSegment.from_ogg(ogg_file)  
    audio = audio.set_channels(1).set_sample_width(2).set_frame_rate(16000)  
    return np.frombuffer(audio.raw_data, np.int16)  

def transcribe_audio(file_path, model):  
    """Расшифровывает аудиофайл и выводит результат."""  
    pcm_array = ogg_to_pcm_array(file_path)  

    rec = KaldiRecognizer(model, 16000)  # Инициализация распознавателя  

    # Обрабатываем данные по частям  
    for i in range(0, len(pcm_array), 4000):  
        data = pcm_array[i:i + 4000].tobytes()  
        if rec.AcceptWaveform(data):  
            result = json.loads(rec.Result())  
            if result.get('text'):  
                print(result['text'])  

    final_result = json.loads(rec.FinalResult())  
    if final_result.get('text'):  
        print(final_result['text']) 
         


In [28]:
record_audio_vosk(WAVE_OUTPUT_FILENAME)

Запись... (нажмите Ctrl+C для остановки)
Остановка записи...
Запись сохранена в output.wav


In [29]:
transcribe_audio(WAVE_OUTPUT_FILENAME, model)

ты поселился в животе


## Потоковый ввод

In [30]:
def stream_audio_and_transcribe_vosk(model):  
    """Потоковая запись аудио с микрофона и его расшифровка в реальном времени."""  
    CHUNK = 1024  # Количество фреймов за один раз  
    FORMAT = pyaudio.paInt16  # Формат звука (16-битный целочисленный)  
    CHANNELS = 1  # Количество каналов (1 - моно, 2 - стерео)  
    RATE = 16000  # Частота дискретизации (в Гц)  

    p = pyaudio.PyAudio()  
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)  

    print("Запись и расшифровка в реальном времени... (нажмите Ctrl+C для остановки)")  

    rec = KaldiRecognizer(model, RATE)  # Инициализация распознавателя  

    results = []  # Список для хранения результатов распознавания  

    try:  
        while True:  
            data = stream.read(CHUNK)  # Чтение данных из потока  
            if rec.AcceptWaveform(data):  # Обработка данных    
                result = json.loads(rec.Result())  
                if 'text' in result and result['text']:  
                    print(result['text'])  # Вывод распознанного текста сразу  
                    results.append(result['text'])  # Добавление текста в список  

            # Обработка части результата для вывод каждого слова  
            partial_result = json.loads(rec.PartialResult())  
            if 'partial' in partial_result and partial_result['partial']:  
                print(partial_result['partial'], end='\r')  # Печать текущего распознаваемого слова  

    except KeyboardInterrupt:  
        print("\nОстановка записи...")  

    finally:  
        stream.stop_stream()  
        stream.close()  
        p.terminate()  

    final_result = json.loads(rec.FinalResult())  
    if final_result.get('text'):  
        print(final_result['text'])  # Печать финального результата  
        results.append(final_result['text'])  # Добавление финального текста в список  

    return results  

In [31]:
transcriptions = stream_audio_and_transcribe_vosk(model) 
result_string = ' '.join(transcriptions)
print(result_string)

Запись и расшифровка в реальном времени... (нажмите Ctrl+C для остановки)
ты поселился в животе
Остановка записи...
ты поселился в животе
ты поселился в животе


## Сохранение в файле и потоковый ввод


In [175]:
def record_and_transcribe_audio_vosk(output_filename, model):  
    """Запись аудио с микрофона и расшифровка в реальном времени."""  
    CHUNK = 1024  # Количество фреймов за один раз  
    FORMAT = pyaudio.paInt16  # Формат звука (16-битный целочисленный)  
    CHANNELS = 1  # Количество каналов (1 - моно)  
    RATE = 16000  # Частота дискретизации (в Гц)  

    p = pyaudio.PyAudio()  
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)  

    print("Запись и расшифровка в реальном времени... (нажмите Ctrl+C для остановки)")  

    rec = KaldiRecognizer(model, RATE)  # Инициализация распознавателя  
    frames = []  # Список для хранения фреймов аудио  
    results = []  # Список для хранения результатов распознавания  

    try:  
        while True:  
            data = stream.read(CHUNK)  # Чтение данных из потока  
            frames.append(data)  # Добавление данных в список  

            if rec.AcceptWaveform(data):  # Обработка данных    
                result = json.loads(rec.Result())  
                if 'text' in result and result['text']:  
                    print(result['text'])  # Вывод распознанного текста сразу  
                    results.append(result['text'])  # Добавление текста в список  

            # Обработка части результата для вывода каждого слова  
            partial_result = json.loads(rec.PartialResult())  
            if 'partial' in partial_result and partial_result['partial']:  
                print(partial_result['partial'], end='\r')  # Печать текущего распознаваемого слова  

    except KeyboardInterrupt:  
        print("\nОстановка записи...")  

    finally:  
        stream.stop_stream()  
        stream.close()  
        p.terminate()  

        # Сохранение аудио в файл  
        with wave.open(output_filename, 'wb') as wf:  
            wf.setnchannels(CHANNELS)  
            wf.setsampwidth(p.get_sample_size(FORMAT))  
            wf.setframerate(RATE)  
            wf.writeframes(b''.join(frames))  # Запись всех фреймов в файл  
        print(f"Запись сохранена в {output_filename}")  

    final_result = json.loads(rec.FinalResult())  
    if final_result.get('text'):  
        print(final_result['text'])  # Печать финального результата  
        results.append(final_result['text'])  # Добавление финального текста в список  

    return results  

In [176]:
results = record_and_transcribe_audio_vosk(WAVE_OUTPUT_FILENAME, model)
result_string = ' '.join(results)
print(result_string)

Запись и расшифровка в реальном времени... (нажмите Ctrl+C для остановки)
и она блять да
до пасхи
нет
как лекс с ним воевать

Остановка записи...
Запись сохранена в output.wav
и она блять да до пасхи нет как лекс с ним воевать


# WISPER

In [5]:
import torch  
import whisper  

model_loaded = whisper.load_model("large")  
model_loaded.load_state_dict(torch.load("whisper_large.pth"))  
output = 'temp_audio.wav'

c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

## Файловый ввод

In [23]:
def record_audio(output_filename):
    """Записывает аудио с микрофона в WAV файл."""
    import pyaudio
    import wave

    CHUNK = 1024  # Количество фреймов за один раз
    FORMAT = pyaudio.paInt32  # Формат звука (32-битный целочисленный)
    CHANNELS = 1  # Количество каналов (1 - моно, 2 - стерео)
    RATE = 44100  # Частота дискретизации (в Гц)

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("Запись... (нажмите Ctrl+C для остановки)")

    frames = []  # Список для хранения фреймов аудио

    try:
        while True:
            data = stream.read(CHUNK)
            frames.append(data)  # Добавление данных в список

    except KeyboardInterrupt:
        print("Остановка записи...")

    stream.stop_stream()
    stream.close()
    p.terminate()

    with wave.open(output_filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))  # Запись всех фреймов в файл

    print(f"Запись сохранена в {output_filename}")

def ogg_to_wav(ogg_file, wav_file):  
    """Конвертирует ogg файл в wav."""  
    if not os.path.exists(ogg_file):  
        raise FileNotFoundError(f"Файл не найден: {ogg_file}")  

    audio = AudioSegment.from_ogg(ogg_file)  
    audio.export(wav_file, format="wav")  

def transcribe_audio(file_path, model):  
    """Расшифровывает аудиофайл и выводит результат."""  
    # Конвертируем ogg в wav, если необходимо  
    if file_path.endswith('.ogg'):  
        wav_file = file_path.replace('.ogg', '.wav')  
        ogg_to_wav(file_path, wav_file)  
        file_path = wav_file  

    # Загружаем аудиофайл  
    audio = whisper.load_audio(file_path)  
    audio = whisper.pad_or_trim(audio)  
    
    result = model.transcribe(file_path, language='ru')  
    print(result['text'])  
    return result
    

In [3]:
  # Пример использования
output_filename = "temp_audio.wav"
record_audio(output_filename)


Запись... (нажмите Ctrl+C для остановки)
Остановка записи...
Запись сохранена в temp_audio.wav


In [10]:
result = transcribe_audio(output_filename, model_loaded)

 Я не хочу тебя торопить, Хочешь молчать, ещё помолчи. Если захочется есть и пить, По батарее три раза стучи.


In [ ]:
out = result['text'] 
out 

# Ру виспер


In [ ]:
import torch  
import whisper  
import huggingface_hub
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline

c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## качаем

In [12]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline

torch_dtype = torch.bfloat16 

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
    setattr(torch.distributed, "is_initialized", lambda : False) # monkey patching
device = torch.device(device)

whisper = WhisperForConditionalGeneration.from_pretrained(
    "antony66/whisper-large-v3-russian", torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True,
  
)

processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vital\.cache\huggingface\hub\models--antony66--whisper-large-v3-russian. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu
c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib

## Грузим

In [ ]:
whisper.save_pretrained('C:/Users/vital/OneDrive/Рабочий стол/vsok и w')  

# Сохранение процессора  
processor.save_pretrained("C:/Users/vital/OneDrive/Рабочий стол/vsok и w")  

## Юзаем

In [29]:
torch_dtype = torch.bfloat16 

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
    setattr(torch.distributed, "is_initialized", lambda : False) # monkey patching
device = torch.device(device)  
whisper = WhisperForConditionalGeneration.from_pretrained("wisp_ru", torch_dtype=torch_dtype)  

# Загрузка процессора  
processor = WhisperProcessor.from_pretrained("wisp_ru")  
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cpu
c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


In [41]:
import os  
import torch  
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline  
import numpy as np  
from pydub import AudioSegment  
from io import BytesIO  

torch_dtype = torch.bfloat16 

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
    setattr(torch.distributed, "is_initialized", lambda : False) # monkey patching
device = torch.device(device)  
whisper = WhisperForConditionalGeneration.from_pretrained("wisp_ru", torch_dtype=torch_dtype)  

# Загрузка процессора  
processor = WhisperProcessor.from_pretrained("wisp_ru")

# Создание пайплайна ASR  
asr_pipeline = pipeline(  
    "automatic-speech-recognition",  
    model=whisper,  
    tokenizer=processor.tokenizer,  
    feature_extractor=processor.feature_extractor,  
    chunk_length_s=30,  
    batch_size=16,  
    return_timestamps=True,  
    torch_dtype=torch_dtype,  
    device=device,  
)  



Device set to use cpu


In [42]:
# Чтение аудиофайла  
output_filename = "recorded_audio.wav"
with open(output_filename, 'rb') as f:  
    wav_data = f.read()  

# Конвертация WAV в Numpy формат  
audio_segment = AudioSegment.from_wav(BytesIO(wav_data))  
numpy_array = np.array(audio_segment.get_array_of_samples())  

# Если стерео, выбираем один канал
if audio_segment.channels == 2:  
    numpy_array = numpy_array[::2]  

# Нормализация аудио
if audio_segment.sample_width == 2:  
    numpy_array = numpy_array.astype(np.float32) / 32768.0  

In [47]:


# Получение транскрипции (без указания forced_decoder_ids)
asr = asr_pipeline(numpy_array,   
                   generate_kwargs={"max_new_tokens": 256},  
                   return_timestamps=False)  

print(asr['text'])

KeyboardInterrupt: 